In [1]:
!nvidia-smi --query-gpu=name --format=csv,noheader | head -n 1

NVIDIA GeForce RTX 4090


In [2]:
import io
import time
import torch
import numpy as np
import PIL
import torchaudio
import matplotlib.pyplot as plt
import cdpam
from IPython.display import Audio
from transformers import EncodecModel, AutoProcessor
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset, Image
from walloc import walloc
from spauq.core.metrics import spauq_eval
class Config: pass

In [3]:
codec = EncodecModel.from_pretrained("facebook/encodec_48khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_48khz")

/home/dgj335/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/dgj335/.local/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type

In [4]:
cdpam_loss = cdpam.CDPAM()

/home/dgj335/.local/lib/python3.10/site-packages/cdpam/cdpam.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(modfolder,map_location="cpu")['state']


In [5]:
MUSDB = load_dataset("danjacobellis/musdb18HQ", split='validation')

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [6]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [7]:
max_duration = 44100*180
def encodec_compress(sample):
    with torch.no_grad():
        x, fs = torchaudio.load(sample['audio']['bytes'])
        x = x[:,:max_duration]
        L = x.shape[-1]
    
        t0 = time.time()
        x_pre = torchaudio.transforms.Resample(fs,processor.sampling_rate)(x)
        x_pre = processor(raw_audio=x_pre, sampling_rate=processor.sampling_rate, return_tensors="pt")
        Y = codec.encode(
            input_values = x_pre['input_values'].to(device),
            padding_mask = x_pre['padding_mask'].to(device),
            bandwidth=12.0,
        )
        Y['audio_codes'] = Y['audio_codes'].to("cpu")
        encode_time = time.time() - t0
    
        t0 = time.time()
        x_hat = codec.decode(
            audio_codes=Y['audio_codes'].to(device),
            audio_scales=Y['audio_scales'],
            padding_mask=x_pre['padding_mask']
        )['audio_values']
        x_hat = torchaudio.transforms.Resample(processor.sampling_rate,fs).to(device)(x_hat)
        x_hat = x_hat[0,:,:L].clamp(-1., 1.)
        decode_time = time.time() - t0
    
        bps = 10*Y['audio_codes'].numel()/(x.numel())
        PSNR = 20*np.log10(2) - 10*np.log10(torch.nn.functional.mse_loss(x,x_hat.to("cpu")))
        SDR = spauq_eval(x,x_hat.to("cpu"),fs=fs)
        SSDR = SDR['SSR']
        SRDR = SDR['SRR']
        cdpam = cdpam_loss.forward(x.to(device),x_hat).mean().item()

    return {
        'compressed': Y,
        'encode_time': encode_time,
        'decode_time': decode_time,
        'bps': bps,
        'L': L,
        'PSNR': PSNR,
        'SSDR': SSDR,
        'SRDR': SRDR,
        'CDPAM': cdpam
    }

In [8]:
device = "cuda"
codec = codec.to(device)
gpu = MUSDB.map(
    encodec_compress,
    writer_batch_size=16,
)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [9]:
def encodec_compress_cpu(sample):
    with torch.no_grad():
        x, fs = torchaudio.load(sample['audio']['bytes'])
        x = x[:,:max_duration]
        L = x.shape[-1]
    
        t0 = time.time()
        x_pre = torchaudio.transforms.Resample(fs,processor.sampling_rate)(x)
        x_pre = processor(raw_audio=x_pre, sampling_rate=processor.sampling_rate, return_tensors="pt")
        Y = codec.encode(
            input_values = x_pre['input_values'].to(device),
            padding_mask = x_pre['padding_mask'].to(device),
            bandwidth=12.0,
        )
        Y['audio_codes'] = Y['audio_codes'].to("cpu")
        encode_time = time.time() - t0
    
        t0 = time.time()
        x_hat = codec.decode(
            audio_codes=Y['audio_codes'].to(device),
            audio_scales=Y['audio_scales'],
            padding_mask=x_pre['padding_mask']
        )['audio_values']
        x_hat = torchaudio.transforms.Resample(processor.sampling_rate,fs).to(device)(x_hat)
        x_hat = x_hat[0,:,:L].clamp(-1., 1.)
        decode_time = time.time() - t0

    return {
        'cpu_encode_time': encode_time,
        'cpu_decode_time': decode_time,
    }

In [10]:
device = "cpu"
codec.eval();
codec = codec.to(device)
cpu = MUSDB.map(encodec_compress_cpu, writer_batch_size=16)
combined = gpu.add_column('cpu_encode_time',cpu['cpu_encode_time'])
combined = combined.add_column('cpu_decode_time',cpu['cpu_decode_time'])

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [11]:
metrics = [
    'encode_time',
    'decode_time',
    'bps',
    'PSNR',
    'SSDR',
    'SRDR',
    'CDPAM',
    'cpu_encode_time',
    'cpu_decode_time',
]

In [12]:
for metric in metrics:
    μ = np.mean(combined[metric])
    print(f"{metric}: {μ}")

encode_time: 1.4809666757583617
decode_time: 1.2616524782180787
bps: 0.1376062073298803
PSNR: 40.74080069732666
SSDR: 24.624387612547174
SRDR: 7.267755789886315
CDPAM: 1.5856438767514192e-05
cpu_encode_time: 5.426530446052551
cpu_decode_time: 5.0741120042800905


In [13]:
combined.push_to_hub("danjacobellis/MUSDB_encodec_12kbps",split='validation')

Uploading the dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/MUSDB_encodec_12kbps/commit/c2ec1f35e260b645ee6bd0a7f8d556a300882b74', commit_message='Upload dataset', commit_description='', oid='c2ec1f35e260b645ee6bd0a7f8d556a300882b74', pr_url=None, pr_revision=None, pr_num=None)